In [1]:
import json
from StringIO import StringIO
import pandas as pd

In [6]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
listings_original = pd.read_csv('Datasources/inside_airbnb/listings.csv')
calendar_original = pd.read_csv('Datasources/inside_airbnb/calendar.csv')

In [3]:
def parse_columns(listings, cols):
    chars = "%$"
    for i in cols:
        listings[i].astype(str).map(lambda x: x.rstrip(chars))
        listings[i] = listings[i].apply(pd.to_numeric, errors='coerce')
        listings[i].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
    return listings  

In [4]:
listings = parse_columns(listings_original, ['host_response_rate', 'cleaning_fee',\
                                     'host_acceptance_rate','extra_people',\
                                     'weekly_price', 'monthly_price', 'security_deposit'])

In [9]:
listings['distance_from_ocean'] = 0
listings['distance_from_ocean'] = listings['distance_from_ocean'].astype('float')
    
for w in listings.index:
    p = float(listings['latitude'][w])
    q = float(listings['longitude'][w])
    lon_diff = (q + 117.235585)*np.pi/180
    lat_diff = (p - 32.802458)*np.pi/180
    a = np.sin(lat_diff/2)**2 + np.cos(p*np.pi/180)*np.cos(32.802458*np.pi/180)*(np.sin(lon_diff/2)**2)
    c = np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = 6371.00*float(c)
    listings['distance_from_ocean'][w] = d

C:\Users\sanka\Anaconda2\envs\py27\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
listings['distance_from_ocean']

0       16.428822
1       14.680150
2       14.615847
3       16.117698
4       16.590429
5       16.591702
6       16.806100
7       16.754990
8       16.785644
9       15.937859
10      15.690154
11      16.167408
12      16.075261
13      16.162247
14      13.746722
15      13.722925
16      13.798799
17      13.389211
18      13.728835
19      14.822134
20      16.671896
21      16.269556
22      14.783864
23      15.103421
24      16.611282
25      14.867815
26      16.116383
27      14.748924
28      15.687380
29      15.091123
          ...    
6578     8.273928
6579     9.149486
6580     9.931912
6581    10.233495
6582    11.654730
6583    15.976335
6584    10.408165
6585     2.886872
6586     2.168124
6587    10.344644
6588    13.607347
6589    16.010018
6590     4.627291
6591     1.178504
6592     8.324562
6593    11.686941
6594     9.900131
6595    18.063823
6596     8.658302
6597    12.858196
6598    11.899211
6599     9.192316
6600    11.076149
6601     8.097011
6602    16